In [ ]:
!pip install selenium google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 20.0 MB/s eta 0:00:00


In [ ]:
import urllib.robotparser
import pandas as pd
import datetime
import time
from selenium.common.exceptions import WebDriverException
from selenium import webdriver
import google_colab_selenium as gs
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.worldweatheronline.com//robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.worldweatheronline.com/da-nang-weather-history/vn.aspx')

True

In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException, NoSuchElementException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

In [ ]:
browser = gs.Chrome(options=chrome_options)
browser.get('https://www.worldweatheronline.com/da-nang-weather-history/vn.aspx')

columns_name = [
    'Date',
    'Time',
    'Weather',
    'Temp',
    'Rain',
    'Cloud',
    'Pressure',
    'Wind',
    'Gust',
]

# Them title vao results
results = [columns_name]

date = datetime.datetime(2025, 3, 4)
end_date = datetime.datetime(2025, 3, 5)
date_col = ""

wait = WebDriverWait(browser, 10)

while date < end_date:
    print(f"Processing date: {date.strftime('%Y-%m-%d')}")
    date_str = date.strftime('%Y-%m-%d')

    try:
        # Adjust date input and submit
        input_date = wait.until(EC.presence_of_element_located((By.ID, 'ctl00_MainContentHolder_txtPastDate')))
        browser.execute_script("arguments[0].value = arguments[1];", input_date, date_str)
        submit_date = wait.until(EC.element_to_be_clickable((By.ID, 'ctl00_MainContentHolder_butShowPastWeather')))
        browser.execute_script("arguments[0].click();", submit_date)
    except (WebDriverException, TimeoutException) as e:
        print(f"Error setting date {date_str}: {str(e)}")
        date += datetime.timedelta(days=1)
        continue

    try:
        # Wait for the date header to load
        date_header = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.days-collapse-date')))
        new_date_col = date_header.text.strip() + ' ' + str(date.year)

        if new_date_col == date_col:
            print(f"Duplicate date detected: {new_date_col}, skipping.")
            date += datetime.timedelta(days=1)
            continue
        date_col = new_date_col
    except TimeoutException:
        print(f"Timeout waiting for date header on {date_str}")
        date += datetime.timedelta(days=1)
        continue

    date += datetime.timedelta(days=1)

    try:
        # Wait for the rows to load
        rows = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'tr.days-details-row1.text-center')))
    except TimeoutException:
        print(f"No data rows found for {date_str}")
        continue


    for row in rows[1:8]:
        try:
            cells = row.find_elements(By.TAG_NAME, 'td')
            time_data = cells[0].find_element(By.TAG_NAME, 'p').text.strip()
            weather = cells[1].find_element(By.TAG_NAME, 'img').get_attribute('title').strip()
            temp = cells[2].find_element(By.CLASS_NAME, 'days-table-forecast-p1').text.replace('\n', ' ').strip()
            rain = cells[3].find_element(By.CLASS_NAME, 'days-rain-number').text.replace('\n', ' ').strip()
            cloud = cells[4].find_element(By.TAG_NAME, 'p').text.strip()
            pressure = cells[5].find_element(By.TAG_NAME, 'p').text.strip()
            wind_speed = cells[6].find_element(By.TAG_NAME, 'div').text.strip()
            gust = cells[7].find_element(By.TAG_NAME, 'div').text.strip()


            results.append([date_col, time_data, weather, temp, rain, cloud, pressure, wind_speed, gust])
        except (NoSuchElementException, IndexError) as e:
            print(f"Error processing row: {str(e)}")
            continue


browser.quit()


<IPython.core.display.Javascript object>

Processing date: 2025-03-04


In [ ]:
df = pd.DataFrame(results)
df

,0,1,2,3,4,5,6,7,8
0,Date,Time,Weather,Temp,Rain,Cloud,Pressure,Wind,Gust
1,"Tuesday, 04 March 2025 2025",00:00,Clear,23 °c,0.0 mm,4%,1006 mb,5 km/h,11 km/h
2,"Tuesday, 04 March 2025 2025",03:00,Mist,21 °c,0.0 mm,46%,1005 mb,6 km/h,11 km/h
3,"Tuesday, 04 March 2025 2025",06:00,Mist,21 °c,0.0 mm,45%,1007 mb,5 km/h,10 km/h
4,"Tuesday, 04 March 2025 2025",09:00,Sunny,25 °c,0.0 mm,7%,1008 mb,3 km/h,3 km/h
5,"Tuesday, 04 March 2025 2025",12:00,Sunny,28 °c,0.0 mm,7%,1006 mb,12 km/h,13 km/h
6,"Tuesday, 04 March 2025 2025",15:00,Sunny,25 °c,0.0 mm,20%,1004 mb,14 km/h,18 km/h
7,"Tuesday, 04 March 2025 2025",18:00,Partly cloudy,23 °c,0.0 mm,26%,1004 mb,5 km/h,8 km/h


In [ ]:
df.to_csv("Weather.csv")